In [1]:
#Change to False if you don't want to retrain the model
train_model = False
#Change to True if you want to save the weights from the model
save_weights = False
epochs = 1 #This model will load previous weights

#Set training & test data path
train_path = 'Input/train-set/*' 
test_path = 'Input/vali-set/*'

In [2]:
import keras
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers import Input
from keras.models import Model
from keras.regularizers import *
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score, precision_recall_fscore_support
from keras.models import model_from_json
from sklearn.metrics import confusion_matrix
from keras.models import model_from_yaml
import numpy as np
import os
import sys
import time
import glob
import numpy as np
import csv
import matplotlib.pyplot as MPL
import cv2
import datetime

Using TensorFlow backend.


In [3]:
d = os.getcwd()
print(d)

C:\Users\Ryan\Desktop\dlass2submission\Code\Algorithm


In [4]:
img_sz = 32
channels = 3

def print_log(s):
    print("{} {}".format(datetime.datetime.now().time(), s))
    return

In [5]:
#obtaining train data

      
Images = glob.glob(train_path)
print_log('Begin')
count = 0
train_X = []
train_Y = []
for img in sorted(Images):
    
    name = int(img[-16:-10][-3:])-1
    full_size_image = cv2.imread(img)
    image = (cv2.resize(full_size_image, (img_sz,img_sz), interpolation=cv2.INTER_CUBIC))
    #gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    gray = image[np.newaxis,:,:]
    x = np.ravel(gray)
    
    train_X.append(gray)
    train_Y.append(name)
    count+=1
    
print(count) 
print(image.shape)
print(gray.shape)
print(x.shape[0])
print_log('Done!!!')

01:08:34.195217 Begin
26
(32, 32, 3)
(1, 32, 32, 3)
3072
01:08:34.216099 Done!!!


In [6]:
#obtaining validation set


Images = glob.glob(test_path)
print_log('Begin')
count = 0
test_X = []
test_Y = []
for img in sorted(Images):
    
    name = int(img[-16:-10][-3:])-1
    full_size_image = cv2.imread(img)
    image = (cv2.resize(full_size_image, (img_sz,img_sz), interpolation=cv2.INTER_CUBIC))
    #gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    tx = np.ravel(image)
    test_X.append(image)
    test_Y.append(int(name))
    count+=1
print(count) 

01:08:34.260902 Begin
26


## Prep Training data to correct format

In [7]:
train_X=np.array(train_X)
train_X.shape = [train_X.shape[0],img_sz,img_sz,channels]
##train_X.shape = [37882,128,128,1]
x_train = train_X

train_Y = np.array(train_Y)
train_Y.shape = [train_Y.shape[0],1]
y_train = train_Y

print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(y_train.shape[0], 'y_train samples')

x_train shape: (26, 32, 32, 3)
y_train shape: (26, 1)
26 train samples
26 y_train samples


## Prep Test data to correct format

In [8]:
test_X=np.array(test_X)
#test_X.shape = [6262,128,128,1]
test_X.shape = [test_X.shape[0],img_sz,img_sz,channels]
x_test = test_X

test_Y = np.array(test_Y)
test_Y.shape = [test_Y.shape[0],1]
y_test = test_Y

print('x_train shape:', x_test.shape)
print('y_train shape:', y_test.shape)
print(x_test.shape[0], 'test samples')
print(y_test.shape[0], 'y_test samples')

x_train shape: (26, 32, 32, 3)
y_train shape: (26, 1)
26 test samples
26 y_test samples


In [9]:
#This sets the conditions of the run for the CNN network.
batch_size = 128
num_classes = 62

data_augmentation = False
x_train.shape[1:]

(32, 32, 3)

In [10]:
#convert label data to shape needed
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## This builds the CNN model structure around the VGG16 model, sets pooling, activations etc

In [11]:
from keras.applications.vgg16 import VGG16
opt = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
initial_model = VGG16(weights='imagenet', include_top=False)
input = Input(shape=(img_sz, img_sz, 3), name='image_input')
x = Flatten()(initial_model(input))
x = Dense(200, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(num_classes,activation='softmax')(x)
model = Model(inputs=input, outputs=x)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,      
              metrics=['acc'])  
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 32, 32, 3)         0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               102600    
_________________________________________________________________
batch_normalization_1 (Batch (None, 200)               800       
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 62)                12462     
Total para

In [12]:
#This loads the weights of the previously trained model
# model.load_weights('Models/model.h5')
# model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

In [13]:
# #Select optimiser and compile model
# opt = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
# model.compile(loss='categorical_crossentropy',
#               optimizer=opt,      
#               metrics=['acc'])  

In [14]:
#Convert values to float and between 0 - 1 
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [15]:
#Run the model, if statement allows to use or not use data augmentation
if train_model:
    total_start_time = time.time()
    if not data_augmentation:
         model.fit(x_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_data=(x_test, y_test),
                  shuffle=True)
    else:
        datagen = ImageDataGenerator(
            zca_whitening=True,  
            rotation_range=0,
            width_shift_range=0.0, 
            height_shift_range=0.0, 
            horizontal_flip=False,
            vertical_flip=False)
        datagen.fit(x_train)

        model.fit_generator(datagen.flow(x_train, y_train,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train.shape[0] // batch_size,
                            epochs=epochs,
    validation_data=(x_test, y_test))
    print('Time Check',(time.time() - total_start_time)/60,'min')

In [16]:
if save_weights:
#serialize model to YAML
    model_yaml = model.to_yaml()
    with open("Models/model_new.yaml", "w") as yaml_file:
        yaml_file.write(model_yaml)    

    # serialize weights to HDF5
    model.save_weights("Models/model_new.h5")
    print("Saved model to disk")

In [17]:
#Evaluate model against test set
score = model.evaluate(x_test, y_test, verbose=1)
print("\nTest accuracy is {}%".format(100.0*score[1]))

26/26 [==============================] - 0s

Test accuracy is 96.15384340286255%


In [18]:
#Predict
preds = model.predict(x_test)
actual = y_test

In [19]:
#flatten to the argmax of the probabilities to choose the option and compare
predict = []
for i in preds:
    predict.append(np.argmax(i))
    #print(np.argmax(i))
print(predict[0])

actual = []
for i in y_test:
    actual.append(np.argmax(i))
    #print(np.argmax(i))
print(actual[0])


0
0


In [20]:
#Create a confusion matrix
conf_matrix = confusion_matrix(actual, predict)
conf_matrix

array([[25,  1],
       [ 0,  0]])

In [21]:
#View the model struture
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 32, 32, 3)         0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               102600    
_________________________________________________________________
batch_normalization_1 (Batch (None, 200)               800       
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 62)                12462     
Total para

In [22]:
#Creates seaborn confusiono matrix

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
def print_confusion_matrix(confusion_matrix, class_names, figsize = (20,14), fontsize=14):
 
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return fig

In [23]:
lab = []
for x in range (0,62):
    lab.append(x)


In [ ]:
print_confusion_matrix(conf_matrix,lab)